## Preliminaries

In [1]:
import pandas as pd
import numpy as np
import PyPDF2
#import textract
import re

## Reading Text

- converted PDF file to txt format for better pre-processing

In [2]:
filename ='JavaBasics-notes.pdf' 

pdfFileObj = open(filename,'rb')               #open allows you to read the file
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   #The pdfReader variable is a readable object that will be parsed
num_pages = pdfReader.numPages                 #discerning the number of pages will allow us to parse through all the pages


count = 0
text = ""
                                                            
while count < num_pages:                       #The while loop will read each page
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
    
#Below if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.

if text != "":
    text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text

else:
    text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

    # Now we have a text variable which contains all the text derived from our PDF file.

In [3]:
text = text.encode('ascii','ignore').lower() #Lowercasing each word

## Extracting Keywords

In [26]:
key = ['inheritance', 'encapsulation', 'multithreading']

In [27]:
keywords = re.findall(str(key),str(text))
len(keywords)                               #Total keywords in document

20600

In [18]:
df = pd.DataFrame(list(set(keywords)),columns=['keywords'])  #Dataframe with unique keywords to avoid repetition in rows

## Calculating Weightage

 - In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. 

- __TF: Term Frequency__, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: 

__TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).__

- __IDF: Inverse Document Frequency__, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: 

__IDF(t) = log_e(Total number of documents / Number of documents with term t in it).__

In [9]:
def weightage(word,text,number_of_documents=1):
    word_list = re.findall(word,text)
    number_of_times_word_appeared =len(word_list)
    tf = number_of_times_word_appeared/float(len(text))
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return number_of_times_word_appeared,tf,idf ,tf_idf    

In [10]:
df['number_of_times_word_appeared'] = df['keywords'].apply(lambda x: weightage(x,str(text))[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,str(text))[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,str(text))[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,str(text))[3])

In [11]:
df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(25)

,keywords,number_of_times_word_appeared,tf,idf,tf_idf


***

## Second Method - Using Gensim library

In [12]:
from gensim.summarization import keywords
import warnings
warnings.filterwarnings("ignore")

In [13]:
values = keywords(text=text,split='\n',scores=True)

In [14]:
data = pd.DataFrame(values,columns=['keyword','score'])
data = data.sort_values('score',ascending=False)
data.head(10)

,keyword,score
0,java basics,0.314014
2,methods,0.247325
1,method,0.247325
3,applets,0.241786
4,applet,0.241786
5,class,0.219800
6,classes,0.219800
7,objects,0.190636
8,object,0.190636
9,programs,0.163243


***

### Third Approach - Using RAKE (Rapid Automatic Keyword Extraction)

In [ ]:
#from rake_nltk import Rake

In [ ]:
#r = Rake()
#r.extract_keywords_from_text(text)

In [ ]:
#phrases = r.get_ranked_phrases_with_scores()

In [ ]:
#table = pd.DataFrame(phrases,columns=['score','Phrase'])
#table = table.sort_values('score',ascending=False)
# table.head(10)

***